Introduction: Safety in schools is a very important requirement and the neighbourhood of the school has a big influence on the school safety.
This project will use Chicago school and crime data from the Course 5 of this certification. School data will be used with FourSquare API to find
the available security( number of Police stations).Depending on the crime counts in the neighbourhood of existing schools,it can be decided if more poice stations are required. For new schools,the crime data analysis and developing a model using machine learning can help with predicting the crime in the neighbourhood in future. The project can be used by any location with the location, school and crime rates data available.This will help the school administration and government bodies to make accurate decisions on which locations are appropriate for new schools as well as which existing school need more security arrangements.


Data Section:  This project will use Chicago school and crime data from the Course 5 of this certification.Only certain columns of inetrest are 
retrieved from the datasets. The records with missing data in the datasets are dropped. Crime data prior to 2013 is used to develop a linear regression model to be used for predicting crime counts for future years.Chicago location data is obtained from the school data. The location co-ordinates for Chicago is obtained using the geopy.geocoders library Nominatim. Maps for the Chicago school data is displayed. FourSquare API will be used to find the police stations near schools to determine the available security and if additional security is required based on the current crime counts.



Installing and Importing of Libraries

In [5]:
try:
    print("Installing BeautifulSoup4...\n")
    !conda install -c conda-forge beautifulsoup4 --yes
    print("BeautifulSoup4 has been successfully installed!\n")
except:
    print("ERROR: could not install BeautifulSoup4!\n")

try:
    print("Installing ProgressBar...\n")
    !conda install -c conda-forge ProgressBar2 --yes
    print("ProgressBar has been successfully installed!\n")
except:
    print("ERROR: could not install ProgressBar!\n")

try:
    print("Installing lxml...\n")
    !conda install -c conda-forge lxml --yes
    print("lxml has been successfully installed!\n")
except:
    print("ERROR: could not install lxml!\n")

try:
    print("Installing GeoPy...\n")
    !conda install -c conda-forge geopy --yes
    print("GeoPy has been successfully installed!\n")
except:
    print("ERROR: could not install GeoPy!\n")

try:
    print("Installing Folium...\n")
    !conda install -c conda-forge folium=0.5.0 --yes
    print("Folium has been successfully installed!\n")
except:
    print("ERROR: could not install Folium!\n")

try:
    print("Importing libraries...\n")
    from progressbar import ProgressBar
    from bs4 import BeautifulSoup as bts # library for web scraping
    import numpy as np # library to handle data in a vectorized manner
    import pandas as pd # library for data analysis
    from pandas.io.json import json_normalize
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import requests # library to handle requests
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import matplotlib as mp # library for visualization
    from sklearn.cluster import KMeans # import k-means from clustering stage
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import folium # map rendering library
    import lxml
    import re
    from time import sleep
    from sklearn import preprocessing
    from sklearn.model_selection import train_test_split
    from sklearn import linear_model
    from sklearn.metrics import r2_score

    print("All libraries imported successfully!\n")
except:
    print("ERROR: Could not import all libraries!\n")

%matplotlib inline

Installing BeautifulSoup4...

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0          conda-forge
    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 -->

Retrieving and Cleaning Chicago Crime Data

In [8]:
# Using the link from Course5 for the Chicago crime dataset
chicago_crime_df = pd.read_csv('https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv')
print("Number of rows and columns in Chicago crime dataset is: ",chicago_crime_df.shape)
chicago_crime_df.head()

Number of rows and columns in Chicago crime dataset is:  (533, 22)


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [9]:
#Creating another dataframe with columns of interest
chicago_crime_df1 = chicago_crime_df[['BLOCK','PRIMARY_TYPE','DESCRIPTION','LOCATION_DESCRIPTION','COMMUNITY_AREA_NUMBER','YEAR','LATITUDE','LONGITUDE']]
print("Number of rows and columns in Chicago crime dataset with selected columns  is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset with selected columns  is:  (533, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,033XX W 66TH ST,THEFT,$500 AND UNDER,ALLEY,66.0,2016,41.773455,-87.706480


In [10]:
#Removing records with missing data
chicago_crime_df1 = chicago_crime_df1.dropna()
chicago_crime_df1 = chicago_crime_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago crime dataset after dropping missing value records is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset after dropping missing value records is:  (487, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,033XX W 66TH ST,THEFT,$500 AND UNDER,ALLEY,66.0,2016,41.773455,-87.706480


In [12]:
#Keeping crime data prior to 2013
chicago_crime_df1 = chicago_crime_df1[chicago_crime_df1['YEAR'] < 2013]
chicago_crime_df1 = chicago_crime_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago crime dataset with years prior to 2013 is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset with years prior to 2013 is:  (357, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,006XX W CHICAGO AVE,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),24.0,2010,41.896447,-87.644939


 Retrieving and Cleaning Chicago School Data

In [13]:
# Using the link from Course5 for the Chicago Public Schools progress report dataset
chicago_school_df = pd.read_csv('https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv')
print("Number of rows and columns in Chicago school report dataset is: ",chicago_school_df.shape)
chicago_school_df.head()


Number of rows and columns in Chicago school report dataset is:  (566, 78)


,School ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,Freshman on Track Rate %,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


In [14]:
#Creating another dataframe with columns of interest
chicago_school_df1 = chicago_school_df[['NAME_OF_SCHOOL','Elementary, Middle, or High School','Street Address','City','State','HEALTHY_SCHOOL_CERTIFIED','AVERAGE_STUDENT_ATTENDANCE','Rate of Misconducts (per 100 students) ','Average Teacher Attendance','Individualized Education Program Compliance Rate ','Latitude','Longitude','COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME']]
print("Number of rows and columns in Chicago school dataset with selected columns  is: ",chicago_school_df1.shape)
chicago_school_df1.head()

Number of rows and columns in Chicago school dataset with selected columns  is:  (566, 14)


,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,HEALTHY_SCHOOL_CERTIFIED,AVERAGE_STUDENT_ATTENDANCE,Rate of Misconducts (per 100 students),Average Teacher Attendance,Individualized Education Program Compliance Rate,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,Yes,96.00%,2.0,96.40%,95.80%,41.924497,-87.644522,7,LINCOLN PARK
1,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,No,95.60%,15.7,95.30%,100.00%,41.760324,-87.556736,43,SOUTH SHORE
2,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,No,95.70%,2.3,94.70%,98.30%,41.747111,-87.731702,70,ASHBURN
3,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,No,95.50%,10.4,95.80%,100.00%,41.809757,-87.672145,61,NEW CITY
4,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,Yes,93.30%,15.6,96.90%,100.00%,41.828146,-87.632794,34,ARMOUR SQUARE


In [15]:
#Removing records with missing data
chicago_school_df1 = chicago_school_df1.dropna()
chicago_school_df1 = chicago_school_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago school dataset after dropping missing value records is: ",chicago_school_df1.shape)
chicago_school_df1.head()

Number of rows and columns in Chicago school dataset after dropping missing value records is:  (565, 14)


,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,HEALTHY_SCHOOL_CERTIFIED,AVERAGE_STUDENT_ATTENDANCE,Rate of Misconducts (per 100 students),Average Teacher Attendance,Individualized Education Program Compliance Rate,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,Yes,96.00%,2.0,96.40%,95.80%,41.924497,-87.644522,7,LINCOLN PARK
1,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,No,95.60%,15.7,95.30%,100.00%,41.760324,-87.556736,43,SOUTH SHORE
2,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,No,95.70%,2.3,94.70%,98.30%,41.747111,-87.731702,70,ASHBURN
3,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,No,95.50%,10.4,95.80%,100.00%,41.809757,-87.672145,61,NEW CITY
4,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,Yes,93.30%,15.6,96.90%,100.00%,41.828146,-87.632794,34,ARMOUR SQUARE


Exploration of the school data: Creating a map of the schools in Chicago School DataFrame

In [16]:
#Creating a map of the schools in Chicago School DataFrame
#Obtaining the geographical co-ordinates of Chicago
address = 'Chicago, USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geographical coordinate of Chicago are 41.8755616, -87.6244212.


In [17]:
# Creating a map using latitude and longitude values
school_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, school, areaname in zip(chicago_school_df1['Latitude'], chicago_school_df1['Longitude'], chicago_school_df1['NAME_OF_SCHOOL'], chicago_school_df1['COMMUNITY_AREA_NAME']):
    label = '{}, {}'.format(school, areaname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(school_map)  

school_map

Exploration of the location data - from the school data

In [18]:
#Obtaining the location data from the school dataframe
chicago_df = chicago_school_df1[['COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME','Latitude','Longitude']]
print("Chicago location dataframe : ",chicago_df.shape)
chicago_df.head()

Chicago location dataframe :  (565, 4)


,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Latitude,Longitude
0,7,LINCOLN PARK,41.924497,-87.644522
1,43,SOUTH SHORE,41.760324,-87.556736
2,70,ASHBURN,41.747111,-87.731702
3,61,NEW CITY,41.809757,-87.672145
4,34,ARMOUR SQUARE,41.828146,-87.632794


Exploration of the crime data

In [19]:
#Creating new Chicago crime dataframe with selected columns from prior Chicago crime dataframe
chicago_crime_df2 = chicago_crime_df1[['COMMUNITY_AREA_NUMBER','YEAR','PRIMARY_TYPE']]
chicago_crime_df2['COMMUNITY_AREA_NUMBER'] = chicago_crime_df2['COMMUNITY_AREA_NUMBER'].astype(int)
print(chicago_crime_df2.shape)
chicago_crime_df2.head()

(357, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,COMMUNITY_AREA_NUMBER,YEAR,PRIMARY_TYPE
0,58,2004,THEFT
1,23,2004,THEFT
2,38,2011,THEFT
3,46,2010,THEFT
4,24,2010,THEFT


In [20]:
chicago_crime_df2 = chicago_crime_df2.groupby(['COMMUNITY_AREA_NUMBER','YEAR','PRIMARY_TYPE']).size().reset_index(name='counts')
print(chicago_crime_df2.shape)
chicago_crime_df2.head()

(340, 4)


,COMMUNITY_AREA_NUMBER,YEAR,PRIMARY_TYPE,counts
0,1,2004,DECEPTIVE PRACTICE,1
1,1,2004,NARCOTICS,1
2,1,2006,CRIMINAL DAMAGE,1
3,1,2006,OTHER OFFENSE,1
4,1,2007,PROSTITUTION,1


In [21]:
#Creating list of area numbers and crime types
areanumber_list = chicago_crime_df2['COMMUNITY_AREA_NUMBER'].tolist()
areanumber_list = list(set(areanumber_list))
crimetype_list = chicago_crime_df2.groupby('PRIMARY_TYPE')['PRIMARY_TYPE'].nunique().index.tolist()
print(areanumber_list)
print(crimetype_list)

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 77]
['ARSON', 'ASSAULT', 'BATTERY', 'BURGLARY', 'CRIM SEXUAL ASSAULT', 'CRIMINAL DAMAGE', 'CRIMINAL TRESPASS', 'DECEPTIVE PRACTICE', 'GAMBLING', 'INTERFERENCE WITH PUBLIC OFFICER', 'INTIMIDATION', 'KIDNAPPING', 'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'NARCOTICS', 'OFFENSE INVOLVING CHILDREN', 'OTHER OFFENSE', 'PROSTITUTION', 'PUBLIC PEACE VIOLATION', 'RITUALISM', 'ROBBERY', 'SEX OFFENSE', 'THEFT', 'WEAPONS VIOLATION']


Feature selection

In [22]:
Feature = chicago_crime_df2[['COMMUNITY_AREA_NUMBER','YEAR','PRIMARY_TYPE']]
Feature_onehot = pd.get_dummies(chicago_crime_df2['PRIMARY_TYPE'])
Feature = Feature.join(Feature_onehot)
Feature = Feature.drop('PRIMARY_TYPE',axis = 1)
Feature.head()

X = Feature
X[0:5]

,COMMUNITY_AREA_NUMBER,YEAR,ARSON,ASSAULT,BATTERY,BURGLARY,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,...,NARCOTICS,OFFENSE INVOLVING CHILDREN,OTHER OFFENSE,PROSTITUTION,PUBLIC PEACE VIOLATION,RITUALISM,ROBBERY,SEX OFFENSE,THEFT,WEAPONS VIOLATION
0,1,2004,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2004,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1,2006,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2006,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,2007,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Label selection

In [23]:
y = chicago_crime_df2['counts'].values
y[0:5]

array([1, 1, 1, 1, 1])

Normalize Data

In [24]:
from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


array([[-1.63212755, -0.86715311, -0.05431254, -0.25657498, -0.44405304,
        -0.24329001, -0.07692308, -0.37030224, -0.17407766,  6.06446847,
        -0.07692308, -0.05431254, -0.05431254, -0.05431254, -0.07692308,
        -0.23643312, -0.36514837, -0.10910895, -0.24329001, -0.09435082,
        -0.07692308, -0.05431254, -0.22222222, -0.09435082, -0.47690337,
        -0.12216944],
       [-1.63212755, -0.86715311, -0.05431254, -0.25657498, -0.44405304,
        -0.24329001, -0.07692308, -0.37030224, -0.17407766, -0.16489491,
        -0.07692308, -0.05431254, -0.05431254, -0.05431254, -0.07692308,
        -0.23643312,  2.73861279, -0.10910895, -0.24329001, -0.09435082,
        -0.07692308, -0.05431254, -0.22222222, -0.09435082, -0.47690337,
        -0.12216944],
       [-1.63212755, -0.22621385, -0.05431254, -0.25657498, -0.44405304,
        -0.24329001, -0.07692308,  2.70049679, -0.17407766, -0.16489491,
        -0.07692308, -0.05431254, -0.05431254, -0.05431254, -0.07692308,
       

In [32]:
#Using training and splitting Linear Regression models to predict the crime counts for each area and printing the R2 Scores

from sklearn.model_selection import train_test_split
from sklearn import linear_model
regr = linear_model.LinearRegression()
from sklearn.metrics import r2_score

r2_score_by_area = pd.DataFrame(columns=['AreaNumber','R2 Score'])
i = 0
for areanumber in areanumber_list:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
    regr.fit(X_train, y_train)
    y_predict = regr.predict(X_test)
    r2_score_by_area.loc[i] = [areanumber,r2_score(y_predict , y_test)]
    coefficients = regr.coef_
    intercept = regr.intercept_
    print("Areanumber: ",areanumber)
    #print("Predicted Crime count: ",y_predict )
    print("Coefficients: ",coefficients)
    print("Intercept: ",intercept)
    i = i +1

print('The mean R2 Score is for the Linear Regressions is', r2_score_by_area['R2 Score'].mean())
r2_score_by_area.head()

Areanumber:  1
Coefficients:  [-7.78822702e-03 -1.81864908e-02 -2.53004312e+12 -8.35650193e+11
 -1.28757552e+12 -7.97349719e+11 -2.65456202e+11 -1.13043879e+12
 -5.86511455e+11 -5.57256035e+11 -2.65456202e+11 -1.94351194e+12
 -1.87983347e+11 -1.87983347e+11 -2.65456202e+11 -7.77291140e+11
 -1.11843346e+12 -3.74299428e+11 -7.97349719e+11 -3.24634825e+11
 -2.65456202e+11 -1.87983347e+11 -7.35109483e+11 -3.24634825e+11
 -1.34874450e+12 -4.17856281e+11]
Intercept:  -222550451053.23215
Areanumber:  2
Coefficients:  [-7.78822702e-03 -1.81864908e-02 -2.53004312e+12 -8.35650193e+11
 -1.28757552e+12 -7.97349719e+11 -2.65456202e+11 -1.13043879e+12
 -5.86511455e+11 -5.57256035e+11 -2.65456202e+11 -1.94351194e+12
 -1.87983347e+11 -1.87983347e+11 -2.65456202e+11 -7.77291140e+11
 -1.11843346e+12 -3.74299428e+11 -7.97349719e+11 -3.24634825e+11
 -2.65456202e+11 -1.87983347e+11 -7.35109483e+11 -3.24634825e+11
 -1.34874450e+12 -4.17856281e+11]
Intercept:  -222550451053.23215
Areanumber:  3
Coefficients:

,AreaNumber,R2 Score
0,1.0,-0.029677
1,2.0,-0.029677
2,3.0,-0.029677
3,4.0,-0.029677
4,5.0,-0.029677


The linear regression model can be used to predict the crime counts in specific locations for future years.